# Optuna 

Run optuna locally on a machine

In [1]:
import optuna

## Simple run of Optuna

Objective function

In [135]:
my_function = lambda x,y: ((x+1)*(y-2))**2 - 2*(x-2) - (y+2)

def objective(trial: optuna.trial.Trial):
    x = trial.suggest_float("x", -20, 20)
    y = trial.suggest_float("y", -20, 20)
    return my_function(x,y)

Study defintion

In [136]:
study = optuna.create_study(study_name="MyStudy", direction="minimize")
#optuna.logging.set_verbosity(optuna.logging.WARN)

[I 2021-08-17 14:36:16,826] A new study created in memory with name: MyStudy


Run the study

In [137]:
study.optimize(objective, n_trials=100, timeout=10)
best_params = study.best_params
print(best_params)

[I 2021-08-17 14:36:19,171] Trial 0 finished with value: 25.567982969073466 and parameters: {'x': -0.8834155765291918, 'y': -16.93036564825089}. Best is trial 0 with value: 25.567982969073466.
[I 2021-08-17 14:36:19,173] Trial 1 finished with value: 26655.514359591965 and parameters: {'x': 8.386957786684487, 'y': 19.403913564025615}. Best is trial 0 with value: 25.567982969073466.
[I 2021-08-17 14:36:19,175] Trial 2 finished with value: 27049.666496832873 and parameters: {'x': 18.564460752200198, 'y': -6.410919825947143}. Best is trial 0 with value: 25.567982969073466.
[I 2021-08-17 14:36:19,177] Trial 3 finished with value: 309.01922417106783 and parameters: {'x': 7.5429227344893235, 'y': 4.114211030858392}. Best is trial 0 with value: 25.567982969073466.
[I 2021-08-17 14:36:19,178] Trial 4 finished with value: 8084.26444245785 and parameters: {'x': 7.70149649128701, 'y': -8.336234629706034}. Best is trial 0 with value: 25.567982969073466.
[I 2021-08-17 14:36:19,182] Trial 5 finished 

{'x': 17.419612386961774, 'y': 1.9542651266384978}


## Store and restart

In [138]:
%%bash
mkdir -p data

In [139]:
db_location = "sqlite:///data/optuna.db"
study_name = "RestartStudy"

In [140]:
%%bash 
rm -f ./data/optuna.db

In [141]:
study = optuna.create_study(study_name=study_name, direction="minimize", storage=db_location)
study.optimize(objective, n_trials=10, timeout=10)
best_params = study.best_params
print(best_params)

[I 2021-08-17 14:37:09,566] A new study created in RDB with name: RestartStudy
[I 2021-08-17 14:37:09,770] Trial 0 finished with value: 30.15617275851475 and parameters: {'x': 11.951112234545729, 'y': 2.5706961325367104}. Best is trial 0 with value: 30.15617275851475.
[I 2021-08-17 14:37:09,991] Trial 1 finished with value: 9209.989144741616 and parameters: {'x': 6.5992151543647, 'y': -10.629153446975636}. Best is trial 0 with value: 30.15617275851475.
[I 2021-08-17 14:37:10,189] Trial 2 finished with value: 24817.52344592532 and parameters: {'x': -9.536793819782448, 'y': -16.439788978777052}. Best is trial 0 with value: 30.15617275851475.
[I 2021-08-17 14:37:10,418] Trial 3 finished with value: 5371.599569381695 and parameters: {'x': 16.760359249094407, 'y': -2.137944224594147}. Best is trial 0 with value: 30.15617275851475.
[I 2021-08-17 14:37:10,656] Trial 4 finished with value: 23875.525499832467 and parameters: {'x': -16.631731220242013, 'y': -7.875900015232723}. Best is trial 0 w

{'x': 11.951112234545729, 'y': 2.5706961325367104}


In [145]:
reloaded_study = optuna.load_study(study_name=study_name, storage=db_location)
study.optimize(objective, timeout=10)
best_params = study.best_params
print(best_params)

[I 2021-08-17 14:38:39,737] Trial 68 finished with value: 74.49642295958166 and parameters: {'x': 0.7960574326079902, 'y': 7.014150974697899}. Best is trial 32 with value: -7.468715749103693.
[I 2021-08-17 14:38:39,873] Trial 69 finished with value: 61.811479692227486 and parameters: {'x': 3.0700360736055723, 'y': 4.055764952584463}. Best is trial 32 with value: -7.468715749103693.
[I 2021-08-17 14:38:40,076] Trial 70 finished with value: 591.7259880758446 and parameters: {'x': 5.417416384947437, 'y': 5.837237519537048}. Best is trial 32 with value: -7.468715749103693.
[I 2021-08-17 14:38:40,244] Trial 71 finished with value: 1.041988305024426 and parameters: {'x': -0.7501553573929713, 'y': 2.4722434595332694}. Best is trial 32 with value: -7.468715749103693.
[I 2021-08-17 14:38:40,385] Trial 72 finished with value: 6.882560965939964 and parameters: {'x': -2.371516829026068, 'y': 1.1666900253840118}. Best is trial 32 with value: -7.468715749103693.
[I 2021-08-17 14:38:40,551] Trial 73 

{'x': 5.226272394930943, 'y': 1.9732425751616103}


In [143]:
optuna.visualization.plot_optimization_history(reloaded_study)

In [146]:
optuna.visualization.plot_contour(study=reloaded_study, params=["x","y"])